In [2]:
#NLP Libraries 
import nltk #natural language toolkit
nltk.download('punkt') #to tokenize individual sentence into words
from nltk.stem.lancaster import LancasterStemmer #importing stemming algo
stemmer= LancasterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#tensorflow and libraries
import tensorflow as tf
import  numpy as np
import tflearn #to implement deep neural network
import random #to generate pseudo random number or shuffle the features
import json #to read json file

In [4]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where are 

In [0]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [0]:
words = []
classes = []
documents = []
ignore = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [0]:
training = []
output = []
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])]) #input layer
net = tflearn.fully_connected(net, 10) #hidden layer
net = tflearn.fully_connected(net, 10) #hidden layer
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax') #ouputlayer
net = tflearn.regression(net) #regression layer

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.03066 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.03066 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.03066 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.03066 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
import pickle #to save all of our data structures 
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [0]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [0]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [14]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))

            results.pop(0)

In [17]:
classify('What are you hours of operation?')

[('hours', 0.98362607)]

In [18]:
response('What are you hours of operation?')

We're open every day 9am-9pm


In [19]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [20]:
#Some of other context free responses.
response('Do you accept Credit Card?')

We accept VISA, Mastercard and AMEX


In [21]:
response('Where can we locate you?')

Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK


In [22]:
response('That is helpful')

Any time!


In [23]:
response('Bye')

Bye! Come back again soon.


In [0]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))

            results.pop(0)

In [25]:
response('Can you please let me know the delivery options?')

We have home delivery options through UBER Eats and Zomato


In [26]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [27]:
context

{'123': 'food'}

In [28]:
response("Hi there!", show_details=True)


context: 
tag: greeting
Hello, thanks for visiting
